In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
moviesData = []
with open('movies.json') as moviesFile:
    for line in moviesFile:
        moviesData.append(json.loads(line))
moviesDF = pd.DataFrame.from_records(moviesData)
moviesDF.head()

,_id,rating,movie_id,description,language,released,rating_cnt,writer,director,cast,img_src,genre,inserted,name
0,{'$oid': '581994c40ad09b0701a0e727'},7.9,tt5286444,Neerja is the story of the courageous Neerja B...,[Hindi],{'$date': '2016-02-19T05:30:00.000+0530'},0,"[Saiwyn Quadras, Sanyukta Shaikh Chawla]",[Ram Madhvani],"[Sonam Kapoor, Shabana Azmi, Yogendra Tikku, A...",https://images-na.ssl-images-amazon.com/images...,"[Biography, Drama, Thriller]",{'$date': '2016-11-02T18:24:52.889+0530'},Neerja
1,{'$oid': '581994c40ad09b0701a0e728'},7.9,tt4434004,A story that revolves around drug abuse in the...,"[Hindi, Panjabi]",{'$date': '2016-06-17T05:30:00.000+0530'},0,"[Sudip Sharma, Abhishek Chaubey]",[Abhishek Chaubey],"[Alia Bhatt, Shahid Kapoor, Diljit Dosanjh, Ka...",https://images-na.ssl-images-amazon.com/images...,"[Crime, Drama, Thriller]",{'$date': '2016-11-02T18:24:52.938+0530'},Udta Punjab
2,{'$oid': '581994c50ad09b0701a0e729'},7.5,tt0248126,Yashvardhan Raichand lives a very wealthy life...,"[Hindi, English, Urdu]",{'$date': '2001-12-14T05:30:00.000+0530'},0,"[Karan Johar, Sheena Parikh]",[Karan Johar],"[Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Kha...",https://images-na.ssl-images-amazon.com/images...,"[Drama, Musical, Romance]",{'$date': '2016-11-02T18:24:53.046+0530'},Kabhi Khushi Kabhie Gham...
3,{'$oid': '581994c50ad09b0701a0e72a'},8.0,tt0347304,"Naina, an introverted, perpetually depressed g...","[Hindi, Urdu, Gujarati, Panjabi]",{'$date': '2003-11-28T05:30:00.000+0530'},0,"[Niranjan Iyengar, Karan Johar]",[Nikkhil Advani],"[Shah Rukh Khan, Preity Zinta, Saif Ali Khan, ...",https://images-na.ssl-images-amazon.com/images...,"[Comedy, Drama, Romance]",{'$date': '2016-11-02T18:24:53.052+0530'},Kal Ho Naa Ho
4,{'$oid': '581994c50ad09b0701a0e72b'},7.6,tt3043252,'Parched' is a story about women set in the he...,"[Hindi, English]",{'$date': '2016-06-17T05:30:00.000+0530'},0,"[Supratik Sen, Leena Yadav]",[Leena Yadav],"[Tannishtha Chatterjee, Radhika Apte, Lehar Kh...",https://images-na.ssl-images-amazon.com/images...,[Drama],{'$date': '2016-11-02T18:24:53.572+0530'},Parched


### I. Plot Based Recommendation System

### 1. Prep the data

In [3]:
plotDF = moviesDF.loc[:, ['name', 'description']]
plotDF.loc[:, 'name'] = plotDF.loc[:, 'name'].apply(lambda x: x.lower())
plotDF.loc[:, 'description'] = plotDF.loc[:, 'description'].apply(lambda x: x.lower())
plotDF = plotDF[plotDF['description'] != '']
plotDF = plotDF.drop_duplicates()
plotDF.head()

,name,description
0,neerja,neerja is the story of the courageous neerja b...
1,udta punjab,a story that revolves around drug abuse in the...
2,kabhi khushi kabhie gham...,yashvardhan raichand lives a very wealthy life...
3,kal ho naa ho,"naina, an introverted, perpetually depressed g..."
4,parched,'parched' is a story about women set in the he...


In [4]:
plotDF.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1751 entries, 0 to 2847
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         1751 non-null   object
 1   description  1751 non-null   object
dtypes: object(2)
memory usage: 41.0+ KB


### 2. Create the TF-IDF matrix

In [5]:
# Import the TfIdfVectorizer from scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF IDF Vectorizer Object
tfidf = TfidfVectorizer(stop_words = 'english')

# Now costruct the TF-IDF Matrix by applying the fit_transform method on the description feature
tfidf_matrix = tfidf.fit_transform(plotDF['description'])

# View the shape of the TfIdf_matrix
tfidf_matrix.shape

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


(1751, 7702)

### 3. Computing the cosine similarity score

In [6]:
# Import linear_kernel from scikit-learn to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix by using linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Let's view the shape of the cosine similarity matrix
print("Shape of cosine similarity matrix: ", cosine_sim.shape)

# Let's quickly view how the matrix looks like in first few column and rows
cosine_sim[0:5, 0:5]

Shape of cosine similarity matrix:  (1751, 1751)


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


array([[1.        , 0.00849819, 0.01322405, 0.02102451, 0.00729834],
       [0.00849819, 1.        , 0.        , 0.        , 0.00940967],
       [0.01322405, 0.        , 1.        , 0.01381885, 0.        ],
       [0.02102451, 0.        , 0.01381885, 1.        , 0.        ],
       [0.00729834, 0.00940967, 0.        , 0.        , 1.        ]])

In [7]:
# Construct a pandas series of movie name as index and index as value
indices = pd.Series(plotDF.index, index = plotDF['name'])
indices.head()

name
neerja                         0
udta punjab                    1
kabhi khushi kabhie gham...    2
kal ho naa ho                  3
parched                        4
dtype: int64

#### 4. Build the recommender function

In [24]:
# Function for returning top 10 movies for a movie title as input
def plot_based_recommender(title, df = plotDF, cosine_sim = cosine_sim, indices = indices):
  # Convert title to lower-case
  title = title.lower()

  # Obtain the index of the movie that matched the title 
  try:
    idx = indices[title]
  except KeyError:
    print('Movie does not exist :(')
    return False

  # Get the pairwise similarity score of all the movies with that movie
  # and convert it into a list of tuples (position, similarity score)
  sim_scores = list(enumerate(cosine_sim[idx]))

  # Sort the movies based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

  # Get the scores of the top 10 most similar movies. Ignore the first movie.
  sim_scores = sim_scores[1:11]

  # get the movie indices
  movie_indices = [sim_score[0] for sim_score in sim_scores]

  # Return the top 10 similar movies
  return df['name'].iloc[movie_indices]

In [25]:
plot_based_recommender('Kahaani')

1


992         subhashchandra
330           socha na tha
44                    ugly
6                       pk
1093          unishe april
1313            saptapadii
50              jab we met
1487             celluloid
2558    adi shankaracharya
1186        chann pardesee
Name: name, dtype: object

## II. Metadata Based Recommender System


### 1.Prep the Data

In [10]:
# Prepare the data
metaDF = moviesDF[['name', 'genre', 'language', 'director', 'cast', 'description']]
metaDF.head()

,name,genre,language,director,cast,description
0,Neerja,"[Biography, Drama, Thriller]",[Hindi],[Ram Madhvani],"[Sonam Kapoor, Shabana Azmi, Yogendra Tikku, A...",Neerja is the story of the courageous Neerja B...
1,Udta Punjab,"[Crime, Drama, Thriller]","[Hindi, Panjabi]",[Abhishek Chaubey],"[Alia Bhatt, Shahid Kapoor, Diljit Dosanjh, Ka...",A story that revolves around drug abuse in the...
2,Kabhi Khushi Kabhie Gham...,"[Drama, Musical, Romance]","[Hindi, English, Urdu]",[Karan Johar],"[Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Kha...",Yashvardhan Raichand lives a very wealthy life...
3,Kal Ho Naa Ho,"[Comedy, Drama, Romance]","[Hindi, Urdu, Gujarati, Panjabi]",[Nikkhil Advani],"[Shah Rukh Khan, Preity Zinta, Saif Ali Khan, ...","Naina, an introverted, perpetually depressed g..."
4,Parched,[Drama],"[Hindi, English]",[Leena Yadav],"[Tannishtha Chatterjee, Radhika Apte, Lehar Kh...",'Parched' is a story about women set in the he...


In [11]:
# We want to keep only the first 3 genre and cast (actors) in the list format
# We want to keep only the first director

metaDF.loc[:, 'genre'] = metaDF.loc[:, 'genre'].apply(lambda x: x[:3] if len(x) > 3 else x)
metaDF.loc[:, 'cast'] = metaDF.loc[:, 'cast'].apply(lambda x: x[:3] if len(x) > 3 else x)
metaDF.loc[:, 'director'] = metaDF.loc[:, 'director'].apply(lambda x: x[:1])
metaDF.head()

/srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,name,genre,language,director,cast,description
0,Neerja,"[Biography, Drama, Thriller]",[Hindi],[Ram Madhvani],"[Sonam Kapoor, Shabana Azmi, Yogendra Tikku]",Neerja is the story of the courageous Neerja B...
1,Udta Punjab,"[Crime, Drama, Thriller]","[Hindi, Panjabi]",[Abhishek Chaubey],"[Alia Bhatt, Shahid Kapoor, Diljit Dosanjh]",A story that revolves around drug abuse in the...
2,Kabhi Khushi Kabhie Gham...,"[Drama, Musical, Romance]","[Hindi, English, Urdu]",[Karan Johar],"[Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Khan]",Yashvardhan Raichand lives a very wealthy life...
3,Kal Ho Naa Ho,"[Comedy, Drama, Romance]","[Hindi, Urdu, Gujarati, Panjabi]",[Nikkhil Advani],"[Shah Rukh Khan, Preity Zinta, Saif Ali Khan]","Naina, an introverted, perpetually depressed g..."
4,Parched,[Drama],"[Hindi, English]",[Leena Yadav],"[Tannishtha Chatterjee, Radhika Apte, Lehar Khan]",'Parched' is a story about women set in the he...


In [12]:
def sanitize(x):
    if isinstance(x, list):
        # Strip spaces
        return [i.replace(" ", "") for i in x]
    else:
        # if it is empty, return an empty string
        if isinstance(x, str):
            return x.replace(" ", "")
        else: 
            return ''

In [13]:
#Apply the generate_list function to cast, keywords, director and genres 
for feature in ['director', 'cast']:
    metaDF[feature] = metaDF[feature].apply(sanitize)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
metaDF.head()

,name,genre,language,director,cast,description
0,Neerja,"[Biography, Drama, Thriller]",[Hindi],[RamMadhvani],"[SonamKapoor, ShabanaAzmi, YogendraTikku]",Neerja is the story of the courageous Neerja B...
1,Udta Punjab,"[Crime, Drama, Thriller]","[Hindi, Panjabi]",[AbhishekChaubey],"[AliaBhatt, ShahidKapoor, DiljitDosanjh]",A story that revolves around drug abuse in the...
2,Kabhi Khushi Kabhie Gham...,"[Drama, Musical, Romance]","[Hindi, English, Urdu]",[KaranJohar],"[AmitabhBachchan, JayaBhaduri, ShahRukhKhan]",Yashvardhan Raichand lives a very wealthy life...
3,Kal Ho Naa Ho,"[Comedy, Drama, Romance]","[Hindi, Urdu, Gujarati, Panjabi]",[NikkhilAdvani],"[ShahRukhKhan, PreityZinta, SaifAliKhan]","Naina, an introverted, perpetually depressed g..."
4,Parched,[Drama],"[Hindi, English]",[LeenaYadav],"[TannishthaChatterjee, RadhikaApte, LeharKhan]",'Parched' is a story about women set in the he...


In [15]:
# Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['genre']) + ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['cast'] + [x['name']] + [x['description']])

# Create the new soup feature 
metaDF['soup'] = metaDF.apply(create_soup, axis=1)

#Display the soup of the first movie 
metaDF.iloc[0]['soup']

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'Biography Drama Thriller RamMadhvani SonamKapoor ShabanaAzmi YogendraTikku Neerja Neerja is the story of the courageous Neerja Bhanot, who sacrificed her life while protecting the lives of 359 passengers on the Pan Am flight 73 in 1986. The flight was hijacked by a terrorist organization.'

In [16]:
# Let's create a dataframe with name and soup
soupDF = metaDF.loc[:, ['name', 'soup']]

# Convert both columns to lowercase
soupDF.loc[:, 'name'] = soupDF.loc[:, 'name'].apply(lambda x: x.lower())
soupDF.loc[:, 'soup'] = soupDF.loc[:, 'soup'].apply(lambda x: x.lower())

# Drop all the rows which are empty
soupDF = soupDF[soupDF['soup'] != '']

# Now drop duplicates in the soupDF
soupDF = soupDF.drop_duplicates()

soupDF.head()
print(soupDF.shape)

(2850, 2)


In [17]:
# Import the TfIdfVectorizer from scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF IDF Vectorizer Object
# with the removal of english stopwords turned on
tfidf = TfidfVectorizer(stop_words = 'english')

# Now costruct the TF-IDF Matrix by applying the fit_transform method on the description feature
tfidf_matrix = tfidf.fit_transform(soupDF['soup'])

# View the shape of the TfIdf_matrix
tfidf_matrix.shape

(2850, 15373)

In [18]:
# Import linear_kernel from scikit-learn to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix by using linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Let's view the shape of the cosine similarity matrix
print("Shape of cosine similarity matrix: ", cosine_sim.shape)

# Let's quickly view how the matrix looks like in first few column and rows
cosine_sim[0:5, 0:5]

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


Shape of cosine similarity matrix:  (2850, 2850)


array([[1.        , 0.01715787, 0.01183263, 0.01769827, 0.00725645],
       [0.01715787, 1.        , 0.00183587, 0.00165802, 0.00912824],
       [0.01183263, 0.00183587, 1.        , 0.04098887, 0.00158062],
       [0.01769827, 0.00165802, 0.04098887, 1.        , 0.0014275 ],
       [0.00725645, 0.00912824, 0.00158062, 0.0014275 , 1.        ]])

In [19]:
# Construct a pandas series of movie name as index and index as value
indices = pd.Series(soupDF.index, index = soupDF['name'])
indices.head()

name
neerja                         0
udta punjab                    1
kabhi khushi kabhie gham...    2
kal ho naa ho                  3
parched                        4
dtype: int64

In [26]:
# Function for returning top 10 movies for a movie title as input
def plot_based_recommender(title, df = soupDF, cosine_sim = cosine_sim, indices = indices):
  # Convert title to lower-case
  title = title.lower()

  # Obtain the index of the movie that matched the title 
  try:
    idx = indices[title]
  except KeyError:
    print('Movie does not exist :(')
    return False

  # Get the pairwise similarity score of all the movies with that movie
  # and convert it into a list of tuples (position, similarity score)
  sim_scores = list(enumerate(cosine_sim[idx]))

  # Sort the movies based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

  # Get the scores of the top 10 most similar movies. Ignore the first movie.
  sim_scores = sim_scores[1:11]

  # get the movie indices
  movie_indices = [sim_score[0] for sim_score in sim_scores]

  # Return the top 10 similar movies
  return df['name'].iloc[movie_indices]

In [28]:
plot_based_recommender('Kahaani')

758                ghatak
323           punjab 1984
44                   ugly
6                      pk
811         baishe srabon
944        macho mustanaa
50             jab we met
1050    open tee bioscope
1620     101 chodhyangal?
854          chaya manush
Name: name, dtype: object